# Data Transformation via Azure Data Factory

As you saw at the end of the previous lesson, different cities use different field names and values to indicate crimes, dates, etc. within their crime data.

For example:
* Some cities use the value "HOMICIDE", "CRIMINAL HOMICIDE" or "MURDER".
* In the New York data, the column is named `offenseDescription` while in the Boston data, the column is named `OFFENSE_CODE_GROUP`.
* In the New York data, the date of the event is in the `reportDate`, while in the Boston data, there is a single column named `MONTH`.

In the case of New York and Boston, here are the unique characteristics of each data set:

| | Offense-Column        | Offense-Value          | Reported-Column  | Reported-Data Type |
|-|-----------------------|------------------------|-----------------------------------|
| New York | `offenseDescription`  | starts with "murder" or "homicide" | `reportDate`     | `timestamp`    |
| Boston | `OFFENSE_CODE_GROUP`  | "Homicide"             | `MONTH`          | `integer`      |

In this notebook, we will use an ADF Databricks Notebooks activity to perform transformations on and extract homicide statistics from the crime data being ingested via ADF.

In this lesson you:
1. Create Databricks Access Token.
2. Add Databricks Notebook activity to pipeline.
3. Connect Copy Activities to Notebook Activity.
4. Publish the updated pipeline.
5. Trigger and Monitor the pipeline run.
6. Verify transformations of data by looking at the generated table in Databricks.
7. Perform a simple aggregation of the data.

## Step 1: Create Databricks Access Token

In this step, you will generate an access token that will be used by ADF to access your Databricks workspace.

In the top right corner of your Databricks workspace (this window), select the Account icon, and then select **User Settings**.
  
  ![Databricks account menu](https://databricksdemostore.blob.core.windows.net/images/03/03/databricks-account-menu.png "Databricks account menu")
  
On the User Settings screen, select Generate New Token.
  
  ![User Settings](https://databricksdemostore.blob.core.windows.net/images/03/03/user-settings.png)

In the Generate New Token dialog, enter a comment, such as "ADF access", leave the Lifetime set to 90 days, and select **Generate**.
  
  ![Generate New Token](https://databricksdemostore.blob.core.windows.net/images/03/03/generate-new-token.png)
  
Copy the generated token and paste it into a text editor, such as Notepad.exe, for use when setting up the Databricks Linked Service in ADF.

> Note: Do not close the dialog containing the generated token until have have save it in a text editor, as it will not be visible again once you close the dialog.
  
  ![Copy Token](https://databricksdemostore.blob.core.windows.net/images/03/03/copy-token.png)

## Step 2: Add Databricks Notebook Activity To Pipeline

ADF provides several Databricks-related activities, including Notebook, Jar, and Python activities, design to allow automated transformation of data using ADF and Databricks. In this exercise, you will add a Databricks Notebook activity to your ADF pipeline which will run the [Databricks-Data-Transformations.dbc]($./includes/Databricks-Data-Transformations) notebook, located in the `/includes` directory of the lab files you uploaded to your Databricks workspace.

Take a few minutes to review the contents of this notebook to understand what transformations are being applied to the datasets being ingested into your Blob storage account via ADF. Also, notice that the notebook defines widgets named `accountKey`, `accountName`, and `containerName`. You will be passing these values in as parameters from ADF so the notebook can connect to your Azure Storage account.

![ADF Widgets](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-widgets.png "ADF Widgets")

To modify your pipeline, return to the ADF UI in a web browser and select the Author (pencil) icon in the left-hand menu, and then select your pipeline under Pipelines.

![ADF UI Author blade](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-author-blade.png "ADF UI Author blade")

Once there, expand Databricks under Activities, then drag the Notebook activity onto the design surface, and drop it to the right of the existing copy activity.

![Add Databricks Notebook activity](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-add-activity-databricks-notebook.png "Add Databricks Notebook activity")

Select the Notebook activity on the design surface to display tabs containing its properties and settings at the bottom of the screen, then enter the following:

### General tab

1. Enter "LabNotebook" into the Name field

  ![Notebook Activity General Tab](https://databricksdemostore.blob.core.windows.net/images/03/03/notebook-activity-general.png)

### Azure Databricks tab

1. Select **+New** next to Databricks Linked Service to create a new Linked Service

  ![Create New Linked Service](https://databricksdemostore.blob.core.windows.net/images/03/03/notebook-activity-azure-databricks-new.png "Create New Linked Service")

2. On the New Linked Service dialog, enter the following:
  - **Name**: enter a name, such as AzureDatabricks. 
  - **Databricks workspace**: Select the workspace you are using for this lab.
  - **Select cluster**: Choose Existing interactive cluster.
  - **Access token**: Paste the token you generated in step one into this field.
  - **Existing cluster id**: Select the Databricks cluster you are using for this lab.
  - Select **Test connection** and ensure you get a Connection successful message.
  - Select **Finish**.
  
  ![New Linked Service (Azure Databricks)](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-linked-service-databricks-notebook.png "New Linked Service (Azure Databricks)")

3. You should now see the AzureDatabricks Linked Service selected in the **Databricks Linked Service** drop down.

  ![Databricks Notebook activity Azure Databricks tab](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-activity-databricks-notebook-databricks-tab.png "Databricks Notebook activity Azure Databricks tab")

### Settings tab

1. **Notebook path**: Enter "/Users/[your-user-account]/03-Data-Ingestion-Via-ADF/includes/Databricks-Data-Transformations" or select Browse, and select the `Databricks-Data-Transformations` notebook from the `/Users/[your-user-account]/03-Data-Ingestion-Via-ADF/includes` folder.
2. Expand Base Parameters.
3. Select **+New**.
4. Enter "accountName" as the parameter Name, and the name of the Storage account you created into the Value.
5. Select **+New** again, and enter "accountKey" as the Name of the second parameter, and paste in the Key for your storage account into the Value.
6. Select **+New** again, and enter "containerName" as the name of the second parameter, and enter "dwtemp" for its Value.

![Databricks Notebook activity Settings tab](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-activity-databricks-notebook-settings.png "Databricks Notebook activity Settings tab")

## Step 3: Connect Copy Activity to Notebook Activity

The final step is to connect the Copy activity with the Notebook activity. Select the small green box on the right-hand side of the copy activity, and drag the arrow onto the Notebook activity on the design surface. Creating this connection sets the output from the copy activity as required input for the Databricks Notebook activity. What this means is that the copy activity has to successfully complete processing and generate its files in your storage account before the Notebook activity runs, ensuring the files required by that Notebook activity are in place at the time of execution.

The finished pipeline should look like the following.

![Complete pipeline](https://databricksdemostore.blob.core.windows.net/images/03/03/data-ingestion-pipeline-complete.png "Complete pipeline")

## Step 4: Publish Pipeline

In the ADF UI toolbar, select **Validate All** to validate the pipeline, and then select **Publish All** to save the changes to the pipeline.

![Publish All](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-publish-all.png "Publish All")

## Step 5: Trigger and Monitor the Pipeline Run

Once the changes have been published, trigger the pipeline by selecting **Trigger**, then **Trigger Now** on your pipeline.

![Trigger ADF Pipeline](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-pipeline-trigger.png "Trigger ADF Pipeline")

Next, navigate to the Monitor screen by selecting the Monitor icon in the left-hand menu.

![Monitor ADF Pipeline](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-monitor-pipeline.png "Monitor ADF Pipeline")

Select the View Activity Runs icon under Actions for the active pipeline run. Here you can observe the ADF pipeline activities running. Notice the Databricks Notebook activity will not run until of the copy activity has completed. Once done, you will see a Succeeded message next to each activity.

![ADF Activity Runs](https://databricksdemostore.blob.core.windows.net/images/03/03/databricks-activity-runs.png "ADF Activity Runs")

To view the notebook output of the run, you can select the Output Action next to the LabNotebook activity, and then select the runPageUrl value. This will open the executed notebook within Databricks, and you can view details of the run in each cell's output.

![Databricks Notebook Output](https://databricksdemostore.blob.core.windows.net/images/03/03/adf-output-databricks-notebook.png "Databricks Notebook output")

-sandbox
## Step 6: Verify transformations of data by looking at the generated table in Databricks

Once the ADF pipleline has finished running, you can check the table by running the following command.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/>Remember to attach your notebook to a cluster before running any cells in your notebook. In the notebook's toolbar, select the drop down arrow next to Detached, then select your cluster under Attach to.

![Attached to cluster](https://databricksdemostore.blob.core.windows.net/images/03/03/databricks-cluster-attach.png)

In [8]:
# Display that list of tables, filtering for users, products, and weblogs tables, to ensure they aren't buried among other tables in the results
sqlContext.tables().filter("tableName = 'homicides_2016'").show()

+--------+---------+-----------+
database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+

You should see the `homicides_2016` tables for `weblogs`, `users` and `products`. You can inspect the data in each table by running the cells below.

In [10]:
%sql
SELECT * FROM homicides_2016 LIMIT 20

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: homicides_2016; line 1 pos 14
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:47)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveRelations$$lookupTableFromCatalog(Analyzer.scala:750)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.resolveRelation(Analyzer.scala:695)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:731)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:724)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:77)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:208)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:349)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:208)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:349)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst

## Step 7: Aggregate the data

Now that you have a normalized dataset containing the data for all cities, you can aggregate the data to compute the number of homicides per month, by city.

Start by creating a new DataFrame containing the homicide data.

In [12]:
homicidesDf = spark.sql("SELECT * FROM homicides_2016")

Perform a simple aggregation to see the number of homicides per month by city.

After running the below query, play around with using Visualizations to represent the data, such as the following:

  ![Databricks visualizations](https://databricksdemostore.blob.core.windows.net/images/03/03/databricks-aggregation-visualization.png "Databricks visualizations")

In [14]:
display(homicidesDf.select("city", "month").orderBy("city", "month").groupBy("city", "month").count())

### Congratulations! 

You have completed the Data Ingestion via ADF module.

In the lab you have learned how to use Azure Data Factory to ingest a remote dataset, and use Azure Databricks activities to address issues with the data, clean and format the data and load it into a global table to support downstream analytics.